In [1]:
import numpy as np
import pandas as pd
import sklearn
df = pd.read_csv(r'D:\Documents\CODES\CS-THESIS\Practice code\Training Data.csv')
df.head()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,1,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,4,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,2,1307923200,Cough Medicine,If you are looking for the secret ingredient i...
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,5,1350777600,Great taffy,Great taffy at a great price. There was a wid...


In [2]:
df = df [["Score", "Text"]]
df = df.rename({"Text" : "Reviews"})
df.head()

,Score,Text
0,5,I have bought several of the Vitality canned d...
1,1,Product arrived labeled as Jumbo Salted Peanut...
2,4,This is a confection that has been around a fe...
3,2,If you are looking for the secret ingredient i...
4,5,Great taffy at a great price. There was a wid...


In [3]:
df['Score'].value_counts()

Score
5    339
4     70
3     37
1     36
2     18
Name: count, dtype: int64

In [4]:
df.shape

(500, 2)

In [6]:
import nltk
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
len(stop_words)

179

In [8]:
negative_words = {'no', 'not', "don't", "aren't", "couldn't", "didn't", "doesn't", "hadn't", "hasn't", "haven't", "isn't", "mightn't", "mustn't", "needn't", "shouldn't", "wasn't", "weren't", "won't", "wouldn't"}
stop_words = set(stop_words)

stop_words -= negative_words
len_stop_words = len(stop_words)

len(stop_words)

160

In [ ]:
import re
from bs4 import BeautifulSoup
import nltk
from nltk.corpus import stopwords

REPLACE_BY_SPACE_RE = re.compile(r'[^\w\s]')
def text_cleaner(text):

    text = BeautifulSoup(text, "html.parser").get_text()
    text = text.lower()
    text = REPLACE_BY_SPACE_RE.sub(' ', text)
    text = re.split(r'\s+', text.strip())
    cleaned_text = " ".join(word for word in text if word not in stop_words)
    return cleaned_text